In [2]:
!pip install python-dotenv gradio groq

In [18]:
# importing libraries
import os
from dotenv import load_dotenv
from groq import Groq
import gradio as gr
import json

In [19]:
# environment variables
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

In [26]:

# Categories and keywords
keywords = {
    "Billing": ["payment", "invoice", "bill", "charge"],
    "Technical Support": ["error", "problem", "not working", "issue", "crash"],
    "General Inquiry": ["hours", "location", "contact"]
}

responses = {
    "Billing": "For billing questions, please check your account settings or contact billing support.",
    "Technical Support": "Please try restarting your device. If the problem persists, contact technical support.",
    "General Inquiry": "Our business hours are 9 AM to 5 PM, Monday to Friday.",
    "Unknown": "I'm not sure I understand. Could you please rephrase your question?"
}

escalation_keywords = ["angry", "frustrated", "not happy", "upset", "annoyed"]

In [27]:
# keyword classification function
def classify_keyword(question):
    question_lower = question.lower()
    scores = {cat: sum(kw in question_lower for kw in kw_list) for cat, kw_list in keywords.items()}
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Unknown", scores

In [28]:
# Escalation detection function
def detect_escalation(question):
    for word in question.lower().split():
        if word in escalation_keywords:
            return True, word
    return False, None

In [35]:
# LLM classifier function
def llm_classify(question):
    prompt = f"""
    You are a helpful assistant that classifies customer support queries into these categories:
    Billing, Technical Support, General Inquiry.

    Return a JSON object with keys: category and reason.
    Question: "{question}"
    Respond only with JSON.
    """
    try:
        response = client.chat.completions.create(
            model="moonshotai/kimi-k2-instruct-0905",
            messages=[
                {"role": "system", "content": "You classify support queries into Billing / Technical Support / General Inquiry."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        text = response.choices[0].message.content.strip()
        return json.loads(text)
    except Exception as e:
        print("LLM classification failed:", str(e))
        return None


In [30]:
def handle_input(question):
    category, scores = classify_keyword(question)
    source = "Keyword"

    if category == "Unknown":
        llm_result = llm_classify(question)
        if llm_result and 'category' in llm_result:
            category = llm_result['category']
            source = "LLM"

    response = responses.get(category, responses["Unknown"])
    escalate, kw = detect_escalation(question)
    escalate_msg = f"Escalation suggested (detected '{kw}')" if escalate else ""

    return {
        "question": question,
        "category": category,
        "source": source,
        "response": response,
        "escalation_msg": escalate_msg
    }


In [37]:
# Gradio interface
def gradio_handler(question):
    result = handle_input(question)
    return result["category"], result["response"], result["source"], result["escalation_msg"]

gr.Interface(
    fn=gradio_handler,
    inputs=[gr.Textbox(lines=2, label="Customer Question", placeholder="Enter your question here...")],
    outputs=[
        gr.Textbox(label="Category"),
        gr.Textbox(label="Response"),
        gr.Textbox(label="Classification Method"),
        # gr.Textbox(label="Escalation Note")
    ],
    title="CUSTOMER SUPPORT CHATBOT",
    description="Handles Billing, Technical Support, or General Inquiry."
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://012d94af8e9be076cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\gradio\blocks.py:1816: UserWarning: A function (gradio_handler) returned too many output values (needed: 3, returned: 4). Ignoring extra values.
    Output components:
        [textbox, textbox, textbox]
    Output values returned:
        ["Technical Support", "Please try restarting your device. If the problem persists, contact technical support.", "LLM", ""]
  warnings.warn(
